In [35]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import sqlite3
import pandas as pd
import os, sys, time, threading
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
from warnings import filterwarnings
import joblib
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from MultiModal.dataset import MultimodalTestDataset
from MultiModal.model import (
   CrossAttention, 
   SoftLabelEncoder,
   ViTFeatureExtractor, 
   ConditionClassifier
)

filterwarnings(action='ignore')

base_path = '/Users/hwangeunbi/Desktop/빅프로젝트/MultiModal_AI_1/data'
# DB 파일 경로 설정
db_path = '/Users/hwangeunbi/Desktop/빅프로젝트/MultiModal_AI_1/sensor_data.db'

def get_db_connection():
    return sqlite3.connect(db_path)

def create_monitoring_table():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS device_status (
        device_id TEXT,
        status TEXT,
        timestamp DATETIME,
        normal_ratio REAL,
        caution_ratio REAL,
        warning_ratio REAL,
        risk_ratio REAL,
        PRIMARY KEY (device_id, timestamp)
    )
    ''')
    conn.commit()
    conn.close()

device = 'cpu'
model_config = {
   'img_dim_h': 120, 'img_dim_w': 160,
   'patch_size': 16, 'embed_dim': 128,
   'num_heads': 4, 'depth': 6,
   'aux_input_dim': 11, 'num_classes': 4
}

def create_monitoring_table():
   conn = sqlite3.connect('sensor_data.db')
   cursor = conn.cursor()
   cursor.execute('''
   CREATE TABLE IF NOT EXISTS device_status (
       device_id TEXT,
       status TEXT,
       timestamp DATETIME,
       normal_ratio REAL,
       caution_ratio REAL,
       warning_ratio REAL,
       risk_ratio REAL,
       PRIMARY KEY (device_id, timestamp)
   )
   ''')
   conn.commit()
   conn.close()

def create_sensor_table():
   conn = sqlite3.connect('sensor_data.db')
   cursor = conn.cursor()
   cursor.execute('''
   CREATE TABLE IF NOT EXISTS sensor_measurements (
       device_id TEXT,
       timestamp DATETIME,
       sensor_name TEXT,
       sensor_value REAL,
       PRIMARY KEY (device_id, timestamp, sensor_name)
   )
   ''')
   conn.commit()
   conn.close()

def update_device_status(device_id, status, normal_ratio, caution_ratio, warning_ratio, risk_ratio):
   conn = sqlite3.connect('sensor_data.db')
   cursor = conn.cursor()
   cursor.execute('''
   INSERT OR REPLACE INTO device_status
   VALUES (?, ?, datetime('now', 'localtime'), ?, ?, ?, ?)
   ''', (device_id, status, normal_ratio, caution_ratio, warning_ratio, risk_ratio))
   conn.commit()
   conn.close()

def insert_sensor_measurements(device_id, window_data, is_first_window=False):
   conn = sqlite3.connect('sensor_data.db')
   cursor = conn.cursor()
   
   data_to_insert = window_data if is_first_window else window_data.tail(30)
   sensor_columns = [col for col in window_data.columns 
                    if col not in ['device_id', 'timestamp', 'filenames']]
   
   timestamp = datetime.now()
   for idx, row in data_to_insert.iterrows():
       current_timestamp = timestamp + timedelta(seconds=idx)
       for sensor in sensor_columns:
           cursor.execute('''
           INSERT OR REPLACE INTO sensor_measurements 
           VALUES (?, ?, ?, ?)
           ''', (device_id, current_timestamp, sensor, float(row[sensor])))
           
   conn.commit()
   conn.close()

def get_latest_status():
   conn = sqlite3.connect('sensor_data.db')
   cursor = conn.cursor()
   cursor.execute('''
   SELECT device_id, status, timestamp, normal_ratio, caution_ratio, warning_ratio, risk_ratio
   FROM device_status
   GROUP BY device_id
   HAVING timestamp = MAX(timestamp)
   ''')
   results = cursor.fetchall()
   conn.close()
   return results

def load_model(device_type):
   model = ConditionClassifier(**model_config)
   path = f'Parameters/{device_type}_Best_State_Model.pth'
   model.load_state_dict(torch.load(path, map_location='cpu'))
   return model.eval()

def load_data():
   agv_data = {}
   for i in range(12, 17):
       with open(f'{base_path}/agv{i}_test_df', 'rb') as file:
           agv_data[i] = joblib.load(file)
           
   oht_data = {}
   for i in range(12, 17):
       with open(f'{base_path}/oht{i}_test_df', 'rb') as file:
           oht_data[i] = joblib.load(file)

   for dataset in [agv_data, oht_data]:
       for key, df in dataset.items():
           df.columns = [col.replace('.', '_') for col in df.columns]
           if 'filenames' in df.columns:
               df['filenames'] = df['filenames'].str.replace('\\', '/', regex=False)
           df.drop(columns=['device_id', 'collection_date', 'collection_time', 'cumulative_operating_day'], 
                   inplace=True)
   
   return agv_data, oht_data

def create_and_populate_tables(data_dict, prefix):
   conn = sqlite3.connect('sensor_data.db')
   for i, df in data_dict.items():
       table_name = f'{prefix}{i}_table'
       df.to_sql(table_name, conn, if_exists='replace', index=False)
       print(f'{table_name}에 데이터 삽입 완료')
   conn.close()

def check_device_status(device_id):
   device_type = 'oht' if 'oht' in device_id.lower() else 'agv'
   number = ''.join(filter(str.isdigit, device_id))
   table = f"{device_type}{number}_table"

   conn = sqlite3.connect('sensor_data.db')
   df = pd.read_sql(f'SELECT * FROM {table}', conn)
   conn.close()

   window_size = 300
   step_size = 30

   for start in range(0, len(df) - window_size + 1, step_size):
       window = df.iloc[start:start + window_size]
       is_first_window = (start == 0)
       
       dataset = MultimodalTestDataset(window)
       dataloader = DataLoader(dataset, batch_size=window_size)
       images, sensors = next(iter(dataloader))

       model = load_model(device_type.upper())
       with torch.no_grad():
           prediction = torch.argmax(model(images, sensors), dim=1)

       risk = (prediction == 3).sum().item()
       warning = (prediction == 2).sum().item()
       caution = (prediction == 1).sum().item()
       normal = (prediction == 0).sum().item()

       total = window_size
       risk_ratio = (risk / total) * 100
       warning_ratio = (warning / total) * 100
       caution_ratio = (caution / total) * 100
       normal_ratio = (normal / total) * 100

       current_status = ""
       if device_type == 'agv':
           if risk >= 20: current_status = "위험"
           elif warning >= 60: current_status = "경고"
           elif caution >= 80: current_status = "주의"
           else: current_status = "정상"
       else:
           if risk >= 15: current_status = "위험"
           elif warning >= 50: current_status = "경고"
           elif caution >= 70: current_status = "주의"
           else: current_status = "정상"

       update_device_status(device_id, current_status, normal_ratio, caution_ratio, warning_ratio, risk_ratio)
       insert_sensor_measurements(device_id, window, is_first_window)

def monitor_device(device_id):
   try:
       while True:
           current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
           print(f"\n[{current_time}] {device_id} 분석 시작")
           check_device_status(device_id)
           time.sleep(30)
   except KeyboardInterrupt:
       return

def monitor_devices(device_ids):
   create_monitoring_table()
   create_sensor_table()
   
   agv_data, oht_data = load_data()
   create_and_populate_tables(agv_data, 'agv')
   create_and_populate_tables(oht_data, 'oht')
   
   threads = []
   for device_id in device_ids:
       thread = threading.Thread(target=monitor_device, args=(device_id,))
       thread.daemon = True
       thread.start()
       threads.append(thread)

   try:
       while True:
           print("\n\n====== 기기 상태 모니터링 ======")
           print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}]")
           
           latest_status = get_latest_status()
           for device_id, status, timestamp, normal, caution, warning, risk in latest_status:
               print(f"\n{device_id}:")
               print(f"상태: {status}")
               print(f"정상: {normal:.1f}%")
               print(f"주의: {caution:.1f}%")
               print(f"경고: {warning:.1f}%")
               print(f"위험: {risk:.1f}%")
           
           time.sleep(30)
   except KeyboardInterrupt:
       print("\n모니터링을 종료합니다...")

app = FastAPI()

app.add_middleware(
   CORSMiddleware,
   allow_origins=["*"],
   allow_credentials=True,
   allow_methods=["*"],
   allow_headers=["*"],
)

@app.get("/device_status/{device_id}")
async def get_device_status(device_id: str):
   conn = sqlite3.connect('sensor_data.db')
   cursor = conn.cursor()

   cursor.execute('''
   SELECT status, timestamp, normal_ratio, caution_ratio, warning_ratio, risk_ratio
   FROM device_status
   WHERE device_id = ?
   ORDER BY timestamp DESC
   LIMIT 1
   ''', (device_id,))

   result = cursor.fetchone()
   conn.close()

   if not result:
       raise HTTPException(status_code=404, detail="Device not found")

   return {
       "current_status": result[0],
       "timestamp": result[1],
       "counts": {
           "normal_count": result[2],
           "caution_count": result[3],
           "warning_count": result[4],
           "risk_count": result[5]
       }
   }

@app.get("/device_history/{device_id}")
async def get_device_history(device_id: str):
   conn = sqlite3.connect('sensor_data.db')

   df = pd.read_sql(f'''
   SELECT timestamp, status, normal_ratio, caution_ratio, warning_ratio, risk_ratio
   FROM device_status
   WHERE device_id = ?
   ORDER BY timestamp DESC
   LIMIT 10
   ''', conn, params=(device_id,))

   conn.close()

   history = []
   for _, row in df.iterrows():
       history.append({
           "timestamp": row["timestamp"],
           "status": row["status"],
           "counts": {
               "normal_count": row["normal_ratio"],
               "caution_count": row["caution_ratio"],
               "warning_count": row["warning_ratio"],
               "risk_count": row["risk_ratio"]
           }
       })

   return history

@app.get("/sensor_data/{device_id}")
async def get_sensor_data(device_id: str):
   conn = sqlite3.connect('sensor_data.db')
   
   df = pd.read_sql(f'''
   SELECT timestampensor_name, sensor_value
   FROM sensor_measurements
   WHERE device_id = ?
   ORDER BY timestamp DESC
   ''', conn, params=(device_id,))
   
   conn.close()
   
   # 센서 데이터를 센서별로 그룹화
   sensor_data = {}
   for sensor in df['sensor_name'].unique():
       sensor_df = df[df['sensor_name'] == sensor]
       sensor_data[sensor] = {
           'timestamps': sensor_df['timestamp'].tolist(),
           'values': sensor_df['sensor_value'].tolist()
       }
       
   return sensor_data

if __name__ == "__main__":
   devices = ['AGV12', 'AGV13', 'AGV14', 'AGV15', 'AGV16']
   monitor_devices(devices)

agv12_table에 데이터 삽입 완료
agv13_table에 데이터 삽입 완료
agv14_table에 데이터 삽입 완료
agv15_table에 데이터 삽입 완료
agv16_table에 데이터 삽입 완료
oht12_table에 데이터 삽입 완료
oht13_table에 데이터 삽입 완료
oht14_table에 데이터 삽입 완료
oht15_table에 데이터 삽입 완료
oht16_table에 데이터 삽입 완료

[2025-01-23 17:53:32] AGV12 분석 시작

[2025-01-23 17:53:32] AGV13 분석 시작

[2025-01-23 17:53:32] AGV14 분석 시작

[2025-01-23 17:53:32] AGV15 분석 시작

[2025-01-23 17:53:32] AGV16 분석 시작


====== 기기 상태 모니터링 ======
[2025-01-23 17:53:32]


====== 기기 상태 모니터링 ======
[2025-01-23 17:54:02]

AGV12:
상태: 정상
정상: 100.0%
주의: 0.0%
경고: 0.0%
위험: 0.0%

AGV13:
상태: 정상
정상: 100.0%
주의: 0.0%
경고: 0.0%
위험: 0.0%

AGV14:
상태: 정상
정상: 100.0%
주의: 0.0%
경고: 0.0%
위험: 0.0%

AGV15:
상태: 정상
정상: 100.0%
주의: 0.0%
경고: 0.0%
위험: 0.0%

AGV16:
상태: 정상
정상: 100.0%
주의: 0.0%
경고: 0.0%
위험: 0.0%

모니터링을 종료합니다...



[2025-01-23 18:12:29] AGV12 분석 시작

[2025-01-23 18:16:44] AGV15 분석 시작
[2025-01-23 18:16:44] AGV16 분석 시작


[2025-01-23 18:16:44] AGV13 분석 시작

[2025-01-23 18:16:52] AGV14 분석 시작

[2025-01-23 18:26:39] AGV12 분석 시작

[2025-01-23 18:36:17] AGV13 분석 시작

[2025-01-23 18:36:17] AGV15 분석 시작

[2025-01-23 18:36:18] AGV16 분석 시작

[2025-01-23 18:36:25] AGV14 분석 시작

[2025-01-23 18:40:44] AGV12 분석 시작

[2025-01-23 18:55:47] AGV13 분석 시작

[2025-01-23 18:55:50] AGV15 분석 시작

[2025-01-23 18:55:58] AGV14 분석 시작

[2025-01-23 18:55:58] AGV16 분석 시작

[2025-01-23 18:55:58] AGV12 분석 시작

[2025-01-23 19:11:32] AGV12 분석 시작

[2025-01-23 19:14:59] AGV13 분석 시작

[2025-01-23 19:15:11] AGV15 분석 시작

[2025-01-23 19:15:21] AGV14 분석 시작

[2025-01-23 19:15:30] AGV16 분석 시작

[2025-01-23 19:25:58] AGV12 분석 시작

[2025-01-23 19:34:18] AGV13 분석 시작

[2025-01-23 19:34:44] AGV15 분석 시작

[2025-01-23 19:34:52] AGV16 분석 시작

[2025-01-23 19:34:52] AGV14 분석 시작

[2025-01-23 19:40:20] AGV12 분석 시작

[2025-01-23 19:53:08] AGV13 분석 시작

[2025-01-23 19:54:0

In [1]:
import sqlite3
import pandas as pd

db_path = '/Users/hwangeunbi/Desktop/빅프로젝트/MultiModal_AI_1/sensor_data.db'

def get_table_data(db_path, table_name):
    conn = sqlite3.connect(db_path)
    query = f"SELECT * FROM {table_name};"
    try:
        df = pd.read_sql(query, conn)
    except Exception as e:
        print(f"오류 발생: {e}")
        df = None
    conn.close()
    return df

# 테이블 데이터 가져오기
table_name = "sensor_measurements"  # 조회하려는 테이블 이름
data = get_table_data(db_path, table_name)

# 데이터 출력
if data is not None:
    print(data.head(10))  # 상위 10개 데이터 출력
else:
    print(f"{table_name} 테이블에서 데이터를 가져올 수 없습니다.")


  device_id                   timestamp     sensor_name  sensor_value
0     AGV12  2025-01-23 17:53:45.198754             NTC         29.60
1     AGV12  2025-01-23 17:53:45.198754           PM1_0         23.00
2     AGV12  2025-01-23 17:53:45.198754           PM2_5         32.00
3     AGV12  2025-01-23 17:53:45.198754            PM10         50.00
4     AGV12  2025-01-23 17:53:45.198754             CT1          1.55
5     AGV12  2025-01-23 17:53:45.198754             CT2         74.85
6     AGV12  2025-01-23 17:53:45.198754             CT3         48.85
7     AGV12  2025-01-23 17:53:45.198754             CT4         19.01
8     AGV12  2025-01-23 17:53:45.198754  ex_temperature         25.00
9     AGV12  2025-01-23 17:53:45.198754     ex_humidity         31.00


In [1]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import sqlite3
import pandas as pd
import os, sys, time, threading
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
from warnings import filterwarnings
import joblib
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from MultiModal.dataset import MultimodalTestDataset
from MultiModal.model import (
  CrossAttention,
  SoftLabelEncoder, 
  ViTFeatureExtractor,
  ConditionClassifier
)

filterwarnings(action='ignore')

base_path = './data'
db_path = './sensor_data.db'

def get_db_connection():
  return sqlite3.connect(db_path)

def create_monitoring_table():
  conn = get_db_connection()
  cursor = conn.cursor()
  cursor.execute('''
  CREATE TABLE IF NOT EXISTS device_status (
      device_id TEXT,
      status TEXT,
      timestamp DATETIME,
      normal_ratio REAL,
      caution_ratio REAL,
      warning_ratio REAL,
      risk_ratio REAL,
      PRIMARY KEY (device_id, timestamp)
  )
  ''')
  conn.commit()
  conn.close()

def create_sensor_table():
  conn = get_db_connection()
  cursor = conn.cursor()
  cursor.execute('''
  CREATE TABLE IF NOT EXISTS sensor_measurements (
      device_id TEXT,
      timestamp DATETIME,
      sensor_name TEXT,
      sensor_value REAL,
      PRIMARY KEY (device_id, timestamp, sensor_name)
  )
  ''')
  conn.commit()
  conn.close()

def create_environment_table():
   conn = get_db_connection()
   cursor = conn.cursor()
   cursor.execute('''
   CREATE TABLE IF NOT EXISTS environment_measurements (
       timestamp DATETIME,
       ex_temperature REAL,
       ex_humidity REAL,
       ex_illuminance REAL,
       PRIMARY KEY (timestamp)
   )
   ''')
   conn.commit()
   conn.close()

device = 'cpu'
model_config = {
  'img_dim_h': 120, 'img_dim_w': 160,
  'patch_size': 16, 'embed_dim': 128,
  'num_heads': 4, 'depth': 6,
  'aux_input_dim': 11, 'num_classes': 4
}

def insert_environment_data(temp, humid, illum):
   conn = get_db_connection()
   cursor = conn.cursor()
   cursor.execute('''
   INSERT OR REPLACE INTO environment_measurements 
   VALUES (datetime('now', 'localtime'), ?, ?, ?)
   ''', (temp, humid, illum))
   conn.commit()
   conn.close()

def update_device_status(device_id, status, normal_ratio, caution_ratio, warning_ratio, risk_ratio):
  conn = get_db_connection()
  cursor = conn.cursor()
  cursor.execute('''
  INSERT OR REPLACE INTO device_status
  VALUES (?, ?, datetime('now', 'localtime'), ?, ?, ?, ?)
  ''', (device_id, status, normal_ratio, caution_ratio, warning_ratio, risk_ratio))
  conn.commit()
  conn.close()

def insert_sensor_measurements(device_id, window_data, is_first_window=False):
   conn = get_db_connection()
   cursor = conn.cursor()
   
   data_to_insert = window_data if is_first_window else window_data.tail(30)
   # 외부 센서 데이터 제외
   sensor_columns = [col for col in window_data.columns 
                    if col not in ['device_id', 'timestamp', 'filenames', 
                                 'ex_temperature', 'ex_humidity', 'ex_illuminance']]
   
   timestamp = datetime.now()
   for idx, row in data_to_insert.iterrows():
       current_timestamp = timestamp + timedelta(seconds=idx)
       for sensor in sensor_columns:
           cursor.execute('''
           INSERT OR REPLACE INTO sensor_measurements 
           VALUES (?, ?, ?, ?)
           ''', (device_id, current_timestamp, sensor, float(row[sensor])))

       # 환경 데이터 저장
       cursor.execute('''
       INSERT OR REPLACE INTO environment_measurements 
       VALUES (?, ?, ?, ?)
       ''', (current_timestamp, float(row['ex_temperature']), 
             float(row['ex_humidity']), float(row['ex_illuminance'])))
           
   conn.commit()
   conn.close()

def get_latest_status():
  conn = get_db_connection()
  cursor = conn.cursor()
  cursor.execute('''
  SELECT device_id, status, timestamp, normal_ratio, caution_ratio, warning_ratio, risk_ratio
  FROM device_status
  GROUP BY device_id
  HAVING timestamp = MAX(timestamp)
  ''')
  results = cursor.fetchall()
  conn.close()
  return results

def load_model(device_type):
  model = ConditionClassifier(**model_config)
  path = f'Parameters/{device_type}_Best_State_Model.pth'
  model.load_state_dict(torch.load(path, map_location='cpu'))
  return model.eval()

def load_data():
  agv_data = {}
  for i in range(12, 17):
      with open(f'{base_path}/agv{i}_test_df', 'rb') as file:
          agv_data[i] = joblib.load(file)
          
  oht_data = {}
  for i in range(12, 17):
      with open(f'{base_path}/oht{i}_test_df', 'rb') as file:
          oht_data[i] = joblib.load(file)

  for dataset in [agv_data, oht_data]:
      for key, df in dataset.items():
          df.columns = [col.replace('.', '_') for col in df.columns]
          if 'filenames' in df.columns:
              df['filenames'] = df['filenames'].str.replace('\\', '/', regex=False)
          df.drop(columns=['device_id', 'collection_date', 'collection_time', 'cumulative_operating_day'], 
                  inplace=True)
  
  return agv_data, oht_data

def create_and_populate_tables(data_dict, prefix):
  conn = get_db_connection()
  for i, df in data_dict.items():
      table_name = f'{prefix}{i}_table'
      df.to_sql(table_name, conn, if_exists='replace', index=False)
      print(f'{table_name}에 데이터 삽입 완료')
  conn.close()

def check_device_status(device_id):
  device_type = 'oht' if 'oht' in device_id.lower() else 'agv'
  number = ''.join(filter(str.isdigit, device_id))
  table = f"{device_type}{number}_table"

  conn = get_db_connection()
  df = pd.read_sql(f'SELECT * FROM {table}', conn)
  conn.close()

  window_size = 300
  step_size = 30

  for start in range(0, len(df) - window_size + 1, step_size):
      window = df.iloc[start:start + window_size]
      is_first_window = (start == 0)
      
      dataset = MultimodalTestDataset(window)
      dataloader = DataLoader(dataset, batch_size=window_size)
      images, sensors = next(iter(dataloader))

      model = load_model(device_type.upper())
      with torch.no_grad():
          prediction = torch.argmax(model(images, sensors), dim=1)

      risk = (prediction == 3).sum().item()
      warning = (prediction == 2).sum().item()
      caution = (prediction == 1).sum().item()
      normal = (prediction == 0).sum().item()

      total = window_size
      risk_ratio = (risk / total) * 100
      warning_ratio = (warning / total) * 100
      caution_ratio = (caution / total) * 100
      normal_ratio = (normal / total) * 100

      current_status = ""
      if device_type == 'agv':
          if risk >= 20: current_status = "위험"
          elif warning >= 60: current_status = "경고"
          elif caution >= 80: current_status = "주의"
          else: current_status = "정상"
      else:
          if risk >= 15: current_status = "위험"
          elif warning >= 50: current_status = "경고"
          elif caution >= 70: current_status = "주의"
          else: current_status = "정상"

      update_device_status(device_id, current_status, normal_ratio, caution_ratio, warning_ratio, risk_ratio)
      insert_sensor_measurements(device_id, window, is_first_window)

def monitor_device(device_id):
  try:
      while True:
          current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
          print(f"\n[{current_time}] {device_id} 분석 시작")
          check_device_status(device_id)
          time.sleep(100)
  except KeyboardInterrupt:
      return

def monitor_devices(device_ids):
  threads = []
  for device_id in device_ids:
      thread = threading.Thread(target=monitor_device, args=(device_id,))
      thread.daemon = True
      thread.start()
      threads.append(thread)

  try:
      while True:
          print("\n\n====== 기기 상태 모니터링 ======")
          print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}]")
          
          latest_status = get_latest_status()
          for device_id, status, timestamp, normal, caution, warning, risk in latest_status:
              print(f"\n{device_id}:")
              print(f"상태: {status}")
              print(f"정상: {normal:.1f}%")
              print(f"주의: {caution:.1f}%")
              print(f"경고: {warning:.1f}%")
              print(f"위험: {risk:.1f}%")
          
          time.sleep(30)
  except KeyboardInterrupt:
      print("\n모니터링을 종료합니다...")

app = FastAPI()

app.add_middleware(
  CORSMiddleware,
  allow_origins=["*"],
  allow_credentials=True,
  allow_methods=["*"],
  allow_headers=["*"],
)

@app.on_event("startup")
async def startup_event():
   create_monitoring_table()
   create_sensor_table() 
   create_environment_table()
   
   agv_data, oht_data = load_data()
   create_and_populate_tables(agv_data, 'agv')
   create_and_populate_tables(oht_data, 'oht')
   
   devices = ['AGV12', 'AGV13', 'AGV14', 'AGV15', 'AGV16']
   monitor_thread = threading.Thread(target=monitor_devices, args=(devices,))
   monitor_thread.daemon = True
   monitor_thread.start()

@app.get("/device_status/{device_id}")
async def get_device_status(device_id: str):
  conn = get_db_connection()
  cursor = conn.cursor()

  cursor.execute('''
  SELECT status, timestamp, normal_ratio, caution_ratio, warning_ratio, risk_ratio
  FROM device_status
  WHERE device_id = ?
  ORDER BY timestamp DESC
  LIMIT 1
  ''', (device_id,))

  result = cursor.fetchone()
  conn.close()

  if not result:
      raise HTTPException(status_code=404, detail="Device not found")

  return {
      "current_status": result[0],
      "timestamp": result[1],
      "counts": {
          "normal_count": result[2],
          "caution_count": result[3],
          "warning_count": result[4],
          "risk_count": result[5]
      }
  }

@app.get("/device_history/{device_id}")
async def get_device_history(device_id: str):
  conn = get_db_connection()

  df = pd.read_sql(f'''
  SELECT timestamp, status, normal_ratio, caution_ratio, warning_ratio, risk_ratio
  FROM device_status
  WHERE device_id = ?
  ORDER BY timestamp DESC
  LIMIT 10
  ''', conn, params=(device_id,))

  conn.close()

  history = []
  for _, row in df.iterrows():
      history.append({
          "timestamp": row["timestamp"],
          "status": row["status"],
          "counts": {
              "normal_count": row["normal_ratio"],
              "caution_count": row["caution_ratio"],
              "warning_count": row["warning_ratio"],
              "risk_count": row["risk_ratio"]
          }
      })

  return history

@app.get("/sensor_data/{device_id}")
async def get_sensor_data(device_id: str):
  conn = get_db_connection()
  
  df = pd.read_sql(f'''
  SELECT timestamp, sensor_name, sensor_value
  FROM sensor_measurements
  WHERE device_id = ?
  ORDER BY timestamp DESC
  LIMIT 900
  ''', conn, params=(device_id,))
  
  conn.close()
  
  sensor_data = {}
  for sensor in df['sensor_name'].unique():
      sensor_df = df[df['sensor_name'] == sensor]
      sensor_data[sensor] = {
          'timestamps': sensor_df['timestamp'].tolist(),
          'values': sensor_df['sensor_value'].tolist()
      }
      
  return sensor_data

@app.get("/environment_data")
async def get_environment_data():
   conn = get_db_connection()
   df = pd.read_sql('''
   SELECT * FROM environment_measurements 
   ORDER BY timestamp DESC 
   LIMIT 900
   ''', conn)
   conn.close()
   
   return {
       "timestamps": df['timestamp'].tolist(),
       "ex_temperature": df['ex_temperature'].tolist(),
       "ex_temperature": df['ex_temperature'].tolist(),
       "ex_temperature": df['ex_temperature'].tolist()
   }


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/envs/em/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/em/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/envs/em/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/opt/anacon

In [2]:
if __name__ == "__main__":
   devices = ['AGV12', 'AGV13', 'AGV14', 'AGV15', 'AGV16']
   monitor_devices(devices)

Exception in thread Exception in thread Thread-5 (monitor_device):
Traceback (most recent call last):
  File "/opt/anaconda3/envs/em/lib/python3.12/site-packages/pandas/io/sql.py", line 2674, in execute
Thread-4 (monitor_device):
Traceback (most recent call last):
  File "/opt/anaconda3/envs/em/lib/python3.12/site-packages/pandas/io/sql.py", line 2674, in execute
Exception in thread Thread-6 (monitor_device):
Traceback (most recent call last):
  File "/opt/anaconda3/envs/em/lib/python3.12/site-packages/pandas/io/sql.py", line 2674, in execute
Exception in thread Thread-7 (monitor_device):
Traceback (most recent call last):
  File "/opt/anaconda3/envs/em/lib/python3.12/site-packages/pandas/io/sql.py", line 2674, in execute
Exception in thread Thread-8 (monitor_device):
Traceback (most recent call last):
  File "/opt/anaconda3/envs/em/lib/python3.12/site-packages/pandas/io/sql.py", line 2674, in execute
    cur.execute(sql, *args)
sqlite3.OperationalError: no such table: agv15_table

The


[2025-01-24 15:49:14] AGV12 분석 시작

[2025-01-24 15:49:14] AGV13 분석 시작

[2025-01-24 15:49:14] AGV14 분석 시작

[2025-01-24 15:49:14] AGV15 분석 시작

[2025-01-24 15:49:14] AGV16 분석 시작


====== 기기 상태 모니터링 ======
[2025-01-24 15:49:14]


OperationalError: no such table: device_status

In [ ]:
import joblib
